In [49]:
from wekeypedia import WikipediaPage as Page

import pandas as pd
from bs4 import BeautifulSoup
import requests

In [6]:
sondage = Page(title='Wikipédia:Sondage/Mention_du_nom_de_naissance_pour_les_personnes_trans', lang="fr")

In [9]:
html = BeautifulSoup(sondage.get_current())

In [19]:
oui_li = (
    html.find(id="Oui_1")
    .parent.find_next_sibling('ol')
    .find_all('li')
)

In [54]:
def extract_user(tag):
    href = tag.find('a').get('href')
    return href if href.startswith('/wiki/Uti') or href.startswith('/wiki/User') else None

oui = (
    pd
    .DataFrame(
        [extract_user(li) for li in oui_li if li.find('a') != None and extract_user(li) != None],
        columns = ['path']
    )
    .assign(
        user = lambda df: df.path.apply(lambda x: requests.utils.unquote(x.split(':')[-1])),
        vote = 'oui',
    )
)

oui

,path,user,vote
0,/wiki/Utilisateur:Sherwood6,Sherwood6,oui
1,/wiki/Utilisatrice:Braveheidi,Braveheidi,oui
2,/wiki/Utilisateur:Caf%C3%A9Buzz,CaféBuzz,oui
3,/wiki/Utilisateur:DarkVador79-UA,DarkVador79-UA,oui
4,/wiki/Utilisateur:JohnNewton8,JohnNewton8,oui
...,...,...,...
162,/wiki/Utilisateur:MorganMauriceFR,MorganMauriceFR,oui
163,/wiki/Utilisateur:Lillian_R%C3%ADnk%C5%AB,Lillian_Rínkū,oui
164,/wiki/Utilisateur:Baidax,Baidax,oui
165,/wiki/Utilisateur:Lutincertain,Lutincertain,oui


In [52]:
non_li = (
    html.find(id="Non_1")
    .parent.find_next_sibling('ol')
    .find_all('li')
)
len(non_li)

161

In [55]:
non = (
    pd
    .DataFrame(
        [extract_user(li) for li in non_li if li.find('a') != None and extract_user(li) != None],
        columns = ['path']
    )
    .assign(
        user = lambda df: df.path.apply(lambda x: requests.utils.unquote(x.split(':')[-1])),
        vote = 'non',
    )
)

non

,path,user,vote
0,/wiki/Utilisateur:Livfornow,Livfornow,non
1,/wiki/Utilisatrice:Kvardek_du,Kvardek_du,non
2,/wiki/Utilisateur:L%C3%A9na,Léna,non
3,/wiki/Utilisateur:Skimel,Skimel,non
4,/wiki/Utilisateur:%C3%86lfgar,Ælfgar,non
...,...,...,...
121,/wiki/Utilisateur:PAC2,PAC2,non
122,/wiki/Utilisateur:Fanfwah,Fanfwah,non
123,/wiki/Utilisateur:Chtfn,Chtfn,non
124,/wiki/Utilisateur:Anejersa,Anejersa,non


In [56]:
neutre_li = (
    html.find(id="Neutre_/_Autre_1")
    .parent.find_next_sibling('ol')
    .find_all('li')
)
len(neutre_li)

14

In [57]:
neutre = (
    pd
    .DataFrame(
        [extract_user(li) for li in neutre_li if li.find('a') != None and extract_user(li) != None],
        columns = ['path']
    )
    .assign(
        user = lambda df: df.path.apply(lambda x: requests.utils.unquote(x.split(':')[-1])),
        vote = 'neutre',
    )
)

neutre

,path,user,vote
0,/wiki/Utilisateur:Slzbg,Slzbg,neutre
1,/wiki/Utilisateur:Turb,Turb,neutre
2,/wiki/Utilisateur:Pa2chant.bis,Pa2chant.bis,neutre
3,/wiki/Utilisateur:Fran%C3%A7oisD,FrançoisD,neutre
4,/wiki/Utilisateur:Borvan53,Borvan53,neutre
5,/wiki/Utilisateur:Toghebon,Toghebon,neutre
6,/wiki/Utilisatrice:Frenouille,Frenouille,neutre
7,/wiki/Utilisatrice:Rartroz,Rartroz,neutre
8,/wiki/Utilisateur:Bischnu,Bischnu,neutre


In [60]:
votes = (
    pd
    .concat([
        oui,
        non,
        neutre
    ])
    .reset_index()
)

votes

,index,path,user,vote
0,0,/wiki/Utilisateur:Sherwood6,Sherwood6,oui
1,1,/wiki/Utilisatrice:Braveheidi,Braveheidi,oui
2,2,/wiki/Utilisateur:Caf%C3%A9Buzz,CaféBuzz,oui
3,3,/wiki/Utilisateur:DarkVador79-UA,DarkVador79-UA,oui
4,4,/wiki/Utilisateur:JohnNewton8,JohnNewton8,oui
...,...,...,...,...
297,4,/wiki/Utilisateur:Borvan53,Borvan53,neutre
298,5,/wiki/Utilisateur:Toghebon,Toghebon,neutre
299,6,/wiki/Utilisatrice:Frenouille,Frenouille,neutre
300,7,/wiki/Utilisatrice:Rartroz,Rartroz,neutre


In [61]:
votes.to_csv('data/votes-q1.csv', index=False)